In [1]:
import sys
sys.path.append('../') 
from imports import *
import cudf

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# Using whole pipeline on GPU using NVIDIA RAPIDS - do later

In [2]:
# Load the saved datasets
X_train = np.load('./fashion-mnist-processed-data/train_data.npy')
X_val = np.load('./fashion-mnist-processed-data/val_data.npy')
X_test = np.load('./fashion-mnist-processed-data/test_data.npy')

y_train = np.load('./fashion-mnist-processed-data/train_targets.npy')
y_val = np.load('./fashion-mnist-processed-data/val_targets.npy')
y_test = np.load('./fashion-mnist-processed-data/test_targets.npy')

In [3]:
# Reshape the training data
n_samples, height, width = X_train.shape
X_train_reshaped = X_train.reshape((n_samples, height*width))

n_val_samples, height_val, width_val = X_val.shape
n_test_samples, height_test, width_test = X_test.shape

# Similarly, reshape the validation and test data if necessary
X_val_reshaped = X_val.reshape((n_val_samples, height_val*width_val))
X_test_reshaped = X_test.reshape((n_test_samples, height_test*width_test))

In [5]:
X_cudf = cudf.DataFrame.from_pandas(pd.DataFrame(X_train_reshaped))

## Logistic Regression without regularization

In [3]:
# Reshape the training data
n_samples, height, width = X_train.shape
X_train_reshaped = X_train.reshape((n_samples, height*width))

n_val_samples, height_val, width_val = X_val.shape
n_test_samples, height_test, width_test = X_test.shape

# Similarly, reshape the validation and test data if necessary
X_val_reshaped = X_val.reshape((n_val_samples, height_val*width_val))
X_test_reshaped = X_test.reshape((n_test_samples, height_test*width_test))

# Setting C to a very high value effectively turns off regularization
log_reg = LogisticRegression(C=1e9, max_iter=1000, solver='saga') # Increase max_iter if the algorithm doesn't converge

# Fit the model
log_reg.fit(X_train_reshaped, y_train)

# Predict on the validation set
y_val_pred = log_reg.predict(X_val_reshaped)

# Evaluate the model on the validation set
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Set Accuracy: {val_accuracy:.4f}")


Validation Set Accuracy: 0.8438


/home/vs/.virtualenvs/torch/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [4]:
print(confusion_matrix(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

[[502   4  14  21   2   1  49   0   7   0]
 [  2 580   3  11   2   0   1   0   1   0]
 [  9   2 450   6  69   1  57   0   6   0]
 [ 22  17  11 493  27   0  28   0   2   0]
 [  1   3  49  20 476   0  49   0   2   0]
 [  0   1   0   0   0 527   0  40   4  28]
 [ 80   5  67  23  69   0 348   0   8   0]
 [  0   0   0   0   0  11   0 565   2  22]
 [  6   2   7   4   4   7   6   4 560   0]
 [  0   0   0   0   0  16   1  20   1 562]]
              precision    recall  f1-score   support

           0       0.81      0.84      0.82       600
           1       0.94      0.97      0.96       600
           2       0.75      0.75      0.75       600
           3       0.85      0.82      0.84       600
           4       0.73      0.79      0.76       600
           5       0.94      0.88      0.91       600
           6       0.65      0.58      0.61       600
           7       0.90      0.94      0.92       600
           8       0.94      0.93      0.94       600
           9       0.92     

## Hyperparameter tuning for logistic regression with no regularization

In [7]:
# Define the model with no regularization
log_reg = LogisticRegression(penalty='none')

# Define the parameter grid
param_grid = {
    'solver': ['lbfgs', 'saga', 'newton-cg', 'sag'],
    'max_iter': [100, 500, 1000, 2000],
    'tol': [1e-4, 1e-3, 1e-2]
}

# Set up the grid search with cross-validation
grid_search = GridSearchCV(log_reg, param_grid, cv=5, verbose=1, n_jobs=-1)

# Perform the grid search
grid_search.fit(X_train_reshaped, y_train)

# Best parameters
print("Best parameters:", grid_search.best_params_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/home/vs/.virtualenvs/torch/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/vs/.virtualenvs/torch/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/vs/.virtualenvs/torch/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/vs/.virtualenvs/torch/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1182: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/vs

KeyboardInterrupt: 